In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
image_size = (240, 240)
batch_size = 24

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Covid19-dataset/train",
    validation_split=0.2,
    subset="training",
    seed=1400,
    image_size=image_size,
    batch_size=batch_size,
)

Found 249 files belonging to 3 classes.
Using 200 files for training.


In [6]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Covid19-dataset/test",
    validation_split=0.2,
    subset="validation",
    seed=1400,
    image_size=image_size,
    batch_size=batch_size,
)

Found 65 files belonging to 3 classes.
Using 13 files for validation.


In [7]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 240, 240, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [8]:
class_names = train_ds.class_names
print(class_names)

['Covid', 'Normal', 'Viral Pneumonia']


In [9]:
data_augmentation = keras.Sequential(
    [
     layers.experimental.preprocessing.RandomFlip("horizontal"),
     layers.experimental.preprocessing.RandomRotation(0.05),
     layers.experimental.preprocessing.RandomZoom(0.10)
    ]
)

In [10]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [11]:
keras.backend.clear_session()

In [12]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    a = data_augmentation(inputs)

    a = layers.experimental.preprocessing.Rescaling(1.0 / 255)(a)
    a = layers.Conv2D(32, 5, strides=2, padding="same")(a)
    a = layers.Activation("relu")(a)
    a = layers.MaxPool2D(3, strides=2, padding="same")(a)

    a = layers.Conv2D(64, 5, padding="same")(a)
    a = layers.Activation("relu")(a)
    a = layers.MaxPool2D(3, strides=2, padding="same")(a)

    for size in [128, 256, 512]:
        a = layers.Activation("relu")(a)
        a = layers.Conv2D(size, 5, padding="same")(a)
        a = layers.MaxPool2D(3, strides=2, padding="same")(a)

    a = layers.Dropout(0.5)(a)
    a = layers.SeparableConv2D(724, 3, padding="same")(a)
    a = layers.Activation("relu")(a)
    a = layers.MaxPool2D(3, strides=2, padding="same")(a)

    a = layers.Flatten()(a)
    a = layers.Dense(256, activation='relu')(a)

    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    a = layers.Dropout(0.2)(a)
    outputs = layers.Dense(units, activation=activation)(a)
    return keras.Model(inputs, outputs)


In [13]:
model = make_model(input_shape=image_size + (3,), num_classes=3)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 240, 240, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 240, 240, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 120, 120, 32)      2432      
                                                                 
 activation (Activation)     (None, 120, 120, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 60, 60, 32)       0         
 )                                                               
                                                             

In [14]:
def scheduler(epoch, lr):
  if epoch < 9:
    return lr
  else:
    return lr * tf.math.exp(-0.008)

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [15]:
epochs = 48

model.compile(optimizer= keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics = ["accuracy"])

history = model.fit(
    train_ds, epochs = epochs, 
    callbacks = [lr_callback],
    validation_data = val_ds)


Epoch 1/48
9/9 [==============================] - 17s 217ms/step - loss: 1.1061 - accuracy: 0.4350 - val_loss: 1.1049 - val_accuracy: 0.3846 - lr: 0.0010
Epoch 2/48
9/9 [==============================] - 5s 65ms/step - loss: 1.0756 - accuracy: 0.4500 - val_loss: 1.0928 - val_accuracy: 0.3846 - lr: 0.0010
Epoch 3/48
9/9 [==============================] - 3s 68ms/step - loss: 0.9914 - accuracy: 0.4500 - val_loss: 0.9830 - val_accuracy: 0.3846 - lr: 0.0010
Epoch 4/48
9/9 [==============================] - 3s 67ms/step - loss: 0.9203 - accuracy: 0.4550 - val_loss: 1.0110 - val_accuracy: 0.5385 - lr: 0.0010
Epoch 5/48
9/9 [==============================] - 3s 68ms/step - loss: 0.8217 - accuracy: 0.5950 - val_loss: 0.9582 - val_accuracy: 0.5385 - lr: 0.0010
Epoch 6/48
9/9 [==============================] - 3s 69ms/step - loss: 0.7467 - accuracy: 0.6450 - val_loss: 1.0900 - val_accuracy: 0.5385 - lr: 0.0010
Epoch 7/48
9/9 [==============================] - 3s 70ms/step - loss: 0.6141 - accura

In [19]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

In [17]:
image_path = "/content/drive/MyDrive/Covid19-dataset/validation/pic1.jpeg"
image_path2 = "/content/drive/MyDrive/Covid19-dataset/validation/pic2.jpeg"
image_path3 = "/content/drive/MyDrive/Covid19-dataset/validation/pic3.jpeg"
image_path4 = "/content/drive/MyDrive/Covid19-dataset/validation/pic4.jpeg"
image_path5 = "/content/drive/MyDrive/Covid19-dataset/validation/pic5.jpeg"
image_path6 = "/content/drive/MyDrive/Covid19-dataset/validation/pic6.jpeg"
image_path7 = "/content/drive/MyDrive/Covid19-dataset/validation/pic7.jpg"
image_path8 = "/content/drive/MyDrive/Covid19-dataset/validation/pic8.jpeg"
image_path9 = "/content/drive/MyDrive/Covid19-dataset/validation/pic9.jpeg"
image_path10 = "/content/drive/MyDrive/Covid19-dataset/validation/pic10.jpeg"

merged_path = [image_path,image_path2,image_path3,image_path4,image_path5,image_path6,image_path7,image_path8,image_path9,
               image_path10]

for img_path in merged_path:
    img = keras.preprocessing.image.load_img(
        img_path, target_size=image_size)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score)))

1/1 [==============================] - 0s 225ms/step
This image most likely belongs to Normal with a 57.27 percent confidence.
1/1 [==============================] - 0s 16ms/step
This image most likely belongs to Covid with a 54.67 percent confidence.
1/1 [==============================] - 0s 15ms/step
This image most likely belongs to Viral Pneumonia with a 57.31 percent confidence.
1/1 [==============================] - 0s 18ms/step
This image most likely belongs to Viral Pneumonia with a 57.61 percent confidence.
1/1 [==============================] - 0s 21ms/step
This image most likely belongs to Viral Pneumonia with a 43.84 percent confidence.
1/1 [==============================] - 0s 16ms/step
This image most likely belongs to Covid with a 57.61 percent confidence.
1/1 [==============================] - 0s 23ms/step
This image most likely belongs to Covid with a 57.61 percent confidence.
1/1 [==============================] - 0s 18ms/step
This image most likely belongs to Viral P